In [1]:
import sys

In [2]:
sys.executable
# for use with
# /usr/local/opt/python@3.8/bin/python3.8 -m pip install ipython-sql

'/usr/local/opt/python@3.8/bin/python3.8'

In [3]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [4]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///chinook.db
Done.


type,name,tbl_name,rootpage,sql
table,album,album,2,"CREATE TABLE [album]( [album_id] INTEGER PRIMARY KEY NOT NULL, [title] NVARCHAR(160) NOT NULL, [artist_id] INTEGER NOT NULL, FOREIGN KEY ([artist_id]) REFERENCES [artist] ([artist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,artist,artist,3,"CREATE TABLE [artist]( [artist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,customer,customer,4,"CREATE TABLE [customer]( [customer_id] INTEGER PRIMARY KEY NOT NULL, [first_name] NVARCHAR(40) NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [company] NVARCHAR(80), [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60) NOT NULL, [support_rep_id] INTEGER, FOREIGN KEY ([support_rep_id]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,employee,employee,5,"CREATE TABLE [employee]( [employee_id] INTEGER PRIMARY KEY NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [first_name] NVARCHAR(20) NOT NULL, [title] NVARCHAR(30), [reports_to] INTEGER, [birthdate] DATETIME, [hire_date] DATETIME, [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60), FOREIGN KEY ([reports_to]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,genre,genre,6,"CREATE TABLE [genre]( [genre_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,invoice,invoice,7,"CREATE TABLE [invoice]( [invoice_id] INTEGER PRIMARY KEY NOT NULL, [customer_id] INTEGER NOT NULL, [invoice_date] DATETIME NOT NULL, [billing_address] NVARCHAR(70), [billing_city] NVARCHAR(40), [billing_state] NVARCHAR(40), [billing_country] NVARCHAR(40), [billing_postal_code] NVARCHAR(10), [total] NUMERIC(10,2) NOT NULL, FOREIGN KEY ([customer_id]) REFERENCES [customer] ([customer_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,invoice_line,invoice_line,8,"CREATE TABLE [invoice_line]( [invoice_line_id] INTEGER PRIMARY KEY NOT NULL, [invoice_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, [unit_price] NUMERIC(10,2) NOT NULL, [quantity] INTEGER NOT NULL, FOREIGN KEY ([invoice_id]) REFERENCES [invoice] ([invoice_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,media_type,media_type,9,"CREATE TABLE [media_type]( [media_type_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist,playlist,10,"CREATE TABLE [playlist]( [playlist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist_track,playlist_track,11,"CREATE TABLE [playlist_track]( [playlist_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, CONSTRAINT [pk_playlist_track] PRIMARY KEY ([playlist_id], [track_id]), FOREIGN KEY ([playlist_id]) REFERENCES [playlist] ([playlist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"


In [5]:
# Tracks by album name; subquery before the main query using a WITH clause

In [6]:
%%sql
WITH track_info AS
 (
     SELECT
        t.name as track_name,
        artist.name as artist,
        album.title album_name,
        t.milliseconds length_milliseconds,
        (t.milliseconds / 1000) length_seconds
        
     FROM track t
     INNER JOIN album ON album.album_id = t.album_id
     INNER JOIN artist ON artist.artist_id = album.artist_id
  )
  
SELECT * FROM track_info
WHERE album_name = "Ride The Lightning";

 * sqlite:///chinook.db
Done.


track_name,artist,album_name,length_milliseconds,length_seconds
Fight Fire With Fire,Metallica,Ride The Lightning,285753,285
Ride The Lightning,Metallica,Ride The Lightning,397740,397
For Whom The Bell Tolls,Metallica,Ride The Lightning,311719,311
Fade To Black,Metallica,Ride The Lightning,414824,414
Trapped Under Ice,Metallica,Ride The Lightning,244532,244
Escape,Metallica,Ride The Lightning,264359,264
Creeping Death,Metallica,Ride The Lightning,396878,396
The Call Of Ktulu,Metallica,Ride The Lightning,534883,534


In [7]:
%%sql

WITH playlist_info AS
  (
     SELECT
        playlist.playlist_id,
        playlist.name playlist_name,
        track.name track_name,
        (track.milliseconds / 1000) length_seconds
     FROM playlist
     LEFT JOIN playlist_track ON playlist_track.playlist_id = playlist.playlist_id
     LEFT JOIN track ON track.track_id = playlist_track.track_id
  )
  
SELECT
    playlist_id,
    playlist_name,
    COUNT(track_name) number_of_tracks,
    SUM(length_seconds) length_seconds
FROM playlist_info
GROUP BY 1, 2
ORDER BY 1;

 * sqlite:///chinook.db
Done.


playlist_id,playlist_name,number_of_tracks,length_seconds
1,Music,3290,876049
2,Movies,0,None
3,TV Shows,213,500987
4,Audiobooks,0,None
5,90’s Music,1477,397970
6,Audiobooks,0,None
7,Movies,0,None
8,Music,3290,876049
9,Music Videos,1,294
10,TV Shows,213,500987


In [8]:
# Creating a custom View first; querying that view right after

In [9]:
%%sql
CREATE VIEW usa_customer_90_dollars AS
    SELECT
        customer.*
    FROM invoice
    INNER JOIN customer ON invoice.customer_id = customer.customer_id
    WHERE country = "USA"
    GROUP BY 1
    HAVING SUM(invoice.total) > 90;

SELECT * FROM usa_customer_90_dollars;


 * sqlite:///chinook.db
(sqlite3.OperationalError) table usa_customer_90_dollars already exists
[SQL: CREATE VIEW usa_customer_90_dollars AS
    SELECT
        customer.*
    FROM invoice
    INNER JOIN customer ON invoice.customer_id = customer.customer_id
    WHERE country = "USA"
    GROUP BY 1
    HAVING SUM(invoice.total) > 90;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [10]:
%%sql
DROP VIEW usa_customer_90_dollars;

 * sqlite:///chinook.db
Done.


[]

In [11]:
# Where regular joins are used to join columns, the union operator is used to join rows from tables and/or views.
# Create two unique views here and make a Union for cheap americans.

In [12]:
%%sql
CREATE VIEW americans AS
    SELECT * FROM customer
    WHERE country = "USA";
    
CREATE VIEW penny_pinchers AS
    SELECT
        customer.*
    FROM invoice
    INNER JOIN customer ON invoice.customer_id = customer.customer_id
    GROUP BY 1
    HAVING SUM(invoice.total) < 20;
    
SELECT * FROM americans

UNION

SELECT * FROM penny_pinchers;

 * sqlite:///chinook.db
(sqlite3.OperationalError) table americans already exists
[SQL: CREATE VIEW americans AS
    SELECT * FROM customer
    WHERE country = "USA";]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [13]:
# %%sql
# DROP VIEW americans;

In [14]:
%%sql

WITH cheap_americans AS
    (
        SELECT * FROM americans
        
        INTERSECT
        
        SELECT * FROM penny_pinchers
    )


SELECT
    employee.first_name || " " || employee.last_name employee_name,
    COUNT(cheapo.customer_id) cheap_americans
FROM employee

LEFT JOIN cheap_americans cheapo ON cheapo.support_rep_id = employee.employee_id

WHERE employee.title = "Sales Support Agent"
GROUP BY 1 ORDER BY 1;

 * sqlite:///chinook.db
Done.


employee_name,cheap_americans
Jane Peacock,0
Margaret Park,0
Steve Johnson,0


In [15]:
# with clauses allow you to define one or more named subqueries. for creating more than one named subquery. 
# To do this, you use a single WITH clause and multiple, comma-separated alias/subquery pairs:

In [16]:
# BEST CUSTOMER IN EACH COUNTRY

In [18]:
%%sql
WITH
    customer_country_purchases AS
        (
         SELECT
             i.customer_id,
             c.country,
             SUM(i.total) total_purchases
         FROM invoice i
         INNER JOIN customer c ON i.customer_id = c.customer_id
         GROUP BY 1, 2
        ),
    country_max_purchase AS
        (
         SELECT
             country,
             MAX(total_purchases) max_purchase
         FROM customer_country_purchases
         GROUP BY 1
        ),
    country_best_customer AS
        (
         SELECT
            cmp.country,
            cmp.max_purchase,
            (
             SELECT ccp.customer_id
             FROM customer_country_purchases ccp
             WHERE ccp.country = cmp.country AND cmp.max_purchase = ccp.total_purchases
            ) customer_id
         FROM country_max_purchase cmp
        )
SELECT
    cbc.country country,
    c.first_name || " " || c.last_name customer_name,
    cbc.max_purchase total_purchased
FROM customer c
INNER JOIN country_best_customer cbc ON cbc.customer_id



 * sqlite:///chinook.db
Done.


country,customer_name,total_purchased
Argentina,Luís Gonçalves,39.6
Argentina,Leonie Köhler,39.6
Argentina,François Tremblay,39.6
Argentina,Bjørn Hansen,39.6
Argentina,František Wichterlová,39.6
Argentina,Helena Holý,39.6
Argentina,Astrid Gruber,39.6
Argentina,Daan Peeters,39.6
Argentina,Kara Nielsen,39.6
Argentina,Eduardo Martins,39.6


In [19]:
# Return each genre, with number of tracks sold in USA by price and percentages

In [28]:
%%sql
WITH
    americans AS
        (
            SELECT * FROM customer WHERE country = "USA"
        ),
    tracks_in_usa AS
        (
            SELECT
                track.name           
            FROM track
            INNER JOIN invoice_line 
                ON invoice_line.track_id = track.track_id
            INNER JOIN invoice
                ON invoice_line.invoice_id = invoice.invoice_id
        )
        
SELECT * FROM tracks_in_usa;







 * sqlite:///chinook.db
Done.


name
Right Next Door to Hell
Dust N' Bones
Live and Let Die
Don't Cry (Original)
Perfect Crime
You Ain't the First
Bad Obsession
Back off Bitch
Double Talkin' Jive
November Rain


In [29]:
# Total dollar amount of sales assigned to each sales support agent

In [ ]:
# Write a query that collates data on purchases from different countries.
# Where a country has only one customer, collect them into an "Other" group.
# The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.
# For each country, include:
# total number of customers
# total value of sales
# average value of sales per customer
# average order value

# $$ ignore the country from the billing address in the invoice table.